![LOGO](../img/MODIN_ver2_hrz.png)

<center><h2>Scale your pandas workflows by changing one line of code</h2>


# Exercise 3: Not Implemented

**GOAL**: Learn what happens when a function is not yet supported in Modin and what functionality is not possible to accelerate

When functionality has not yet been implemented, we default to pandas

![](../img/convert_to_pandas.png)

We convert a Modin dataframe to pandas to do the operation, then convert it back once it is finished. These operations will have a high overhead due to the communication involved and will take longer than pandas.

When this is happening, a warning will be given to the user to inform them that this operation will take longer than usual. For example, `DataFrame.kurtosis` is not yet implemented. In this case, when a user tries to use it, they will see this warning:

```
UserWarning: `DataFrame.kurtosis` defaulting to pandas implementation.
```

## Concept for exercise: Default to pandas

In this section of the exercise we will see first-hand how the runtime is affected by operations that are not implemented.

In [ ]:
import modin.pandas as pd
import pandas
import numpy as np
import time

frame_data = np.random.randint(0, 100, size=(2**18, 2**8))
df = pd.DataFrame(frame_data).add_prefix("col")

In [ ]:
pandas_df = pandas.DataFrame(frame_data).add_prefix("col")

In [ ]:
modin_start = time.time()

print(df.kurtosis())

modin_end = time.time()
print("Modin kurtosis took {} seconds.".format(round(modin_end - modin_start, 4)))

In [ ]:
pandas_start = time.time()

print(pandas_df.kurtosis())

pandas_end = time.time()
print("pandas kurtosis took {} seconds.".format(round(pandas_end - pandas_start, 4)))

## Concept for exercise: Register custom functions

Modin's user-facing API is pandas, but it is possible that we do not yet support your favorite or most-needed functionalities. Your user-defined function may also be able to be executed more efficiently if you pre-define the type of function it is (e.g. map, reduction, etc.). To solve either case, it is possible to register a custom function to be applied to your data.

### Registering a custom function for all query compilers

To register a custom function for a query compiler, we first need to import it:

```python
from modin.backends.pandas.query_compiler import PandasQueryCompiler
```

The `PandasQueryCompiler` is responsible for defining and compiling the queries that can be operated on by Modin, and is specific to the pandas backend. Any queries defined here must also both be compatible with and result in a `pandas.DataFrame`. Many functionalities are very simply implemented, as you can see in the current code: [Link](https://github.com/modin-project/modin/blob/f15fb8ea776ed039893130b1e85053e875912d4b/modin/backends/pandas/query_compiler.py#L365).

If we want to register a new function, we next to understand what kind of function it is. In our example, we will use `kurtosis`, which is a reduction. So we next want to import the function type so we can use it in our definition:

```python
from modin.data_management.functions import ReductionFunction
```

Then we can just use the `ReductionFunction.register` `classmethod` and assign it to the `PandasQueryCompiler`:

```python
PandasQueryCompiler.kurtosis = ReductionFunction.register(pandas.DataFrame.kurtosis)
```

Finally, we want a handle to it from the `DataFrame`, so we need to create a way to do that:

```python
def kurtosis_func(self, **kwargs):
    # The constructor allows you to pass in a query compiler as a keyword argument
    return self.__constructor__(query_compiler=self._query_compiler.kurtosis(**kwargs))

pd.DataFrame.kurtosis_custom = kurtosis_func
```

And then you can use it like you usually would:

```python
df.kurtosis_custom()
```

In [ ]:
from modin.backends.pandas.query_compiler import PandasQueryCompiler
from modin.data_management.functions import ReductionFunction

In [ ]:
PandasQueryCompiler.kurtosis_custom = ReductionFunction.register(pandas.DataFrame.kurtosis)

In [ ]:
# The function signature came from the pandas documentation:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.kurtosis.html
def kurtosis_func(self, axis=None, skipna=None, level=None, numeric_only=None, **kwargs):
    # We need to specify the axis for the backend
    if axis is None:
        axis = 0
    # The constructor allows you to pass in a query compiler as a keyword argument
    # Reduce dimension is used for reductions
    # We also pass all keyword arguments here to ensure correctness
    return self._reduce_dimension(
        self._query_compiler.kurtosis_custom(
            axis=axis, skipna=skipna, level=level, numeric_only=numeric_only, **kwargs
        )
    )

pd.DataFrame.kurtosis_custom = kurtosis_func

In [ ]:
start = time.time()

print(df.kurtosis())

end = time.time()
print("Modin kurtosis took {} seconds.".format(end - start))

In [ ]:
start = time.time()

print(df.kurtosis_custom())

end = time.time()
print("Modin kurtosis_custom took {} seconds.".format(end - start))

## Congratulations! You have just implemented `DataFrame.kurtosis`!

## Consider opening a pull request: https://github.com/modin-project/modin/pulls

For a complete list of what is implemented, see the [documentation](https://modin.readthedocs.io/en/latest/UsingPandasonRay/dataframe_supported.html)

## Test your knowledge: Add a custom function for another reduction: `DataFrame.mad`

See the pandas documentation for the correct signature: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.mad.html

In [ ]:
modin_mad_start = time.time()

# Implement your function here! Put the result of your custom `mad` in the variable `modin_mad`
# Hint: Look at the kurtosis walkthrough above
modin_mad = ...

modin_mad_end = time.time()

In [ ]:
# Evaluation code, do not change!
pandas_mad_start = time.time()
pandas_mad = pandas_df.mad()
pandas_mad_end = time.time()

assert isinstance(modin_mad, pd.Series), "This is not a distributed Modin object, try again"
assert pandas_mad_end - pandas_mad_start > modin_mad_end - modin_mad_start, \
    "Your implementation was too slow, or you used the defaulting to pandas approach. Try again"
assert modin_mad._to_pandas().equals(pandas_mad), "Your result did not match the result of pandas, tray again"

### Now that you are able to create custom functions, you know enough to contribute to Modin!